# Integer numbers
$x\in\mathbb{Z}$
## Python disclaimer
Originally computers we designed to solve computational tasks (over numbers). CPU Architectures (e.g. x64) encount the fact that operations over $\mathbb{Z}$ and $\mathbb{R}$ should be predictibly fast: they [fix number representation length](https://en.wikipedia.org/wiki/Processor_register#Size), sacrifising precision or edge cases.

Most programming languages reuse CPU number representation, defined in [standards](https://en.wikipedia.org/wiki/IEEE_754-2008_revision): `short, int, long, float/real, double` in majority of languages are binary interchangable.

Python is following this strategy for $\mathbb{R}$ (`float`) data, but disrespects for $\mathbb{Z}$ (`int`).

In [1]:
import sys
x = 10000000000000000000000000000000000000000000000000000000000000000000000000000000000
y = 20
a = 20.
b = 0.000000000000000000000000000000000000000000000000000000000000000000000000000000001
print(f'{type(x)}\t x ~ {sys.getsizeof(x)} B; y ~ {sys.getsizeof(y)} B')
print(f'{type(a)}\t a ~ {sys.getsizeof(a)} B; b ~ {sys.getsizeof(b)} B')

<class 'int'>	 x ~ 64 B; y ~ 28 B
<class 'float'>	 a ~ 24 B; b ~ 24 B


This is how this difference can be observed

In [2]:
print("x^4 =", x**4)
print("b^4 =", b**4, b**4 == 0)

x^4 = 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
b^4 = 0.0 True


To make python work as other languages we will use [native types](https://stackoverflow.com/questions/19716315/how-to-create-a-fixed-size-unsigned-integer-in-python).

In [3]:
from ctypes import c_char, c_byte, c_ushort, c_int16, c_uint32, c_int32, c_float
import ctypes

i = c_int32(2000000000)
f = c_float()
f.value = .4

## Signed integer numbers

Binary number representation:

47<sub>10</sub> = 
**1** * 2<sup>5</sup> + 0 * 2<sup>4</sup> + **1** * 2<sup>3</sup> + **1** * 2<sup>2</sup> + **1** * 2<sup>1</sup> + **1** * 2<sup>0</sup> 
= 101111<sub>2</sub> = 00101111 (char) = 00000000 00000000 00000000 00101111 (int)
    
```
[high bit, most significant bit = msb] 00000000 00000000 00000000 0 0 1 0 1 1 1 1 [low bit, lsb]
                                      31                          7 6 5 4 3 2 1 0
```

In [4]:
i = c_int32(0b101111)
print(i.value)

47


## What about negatives?

### Sign-magnitude
Use the **MSB** to store negative sign (1 ~ -). Keep others to represent a magnitude.

$-10_{10} = -00001010_2=10001010_2$

- What about simple arithmetics?
- Two zeros?

### One's complement
Use the **MSB** to store negative sign (1 ~ -). Invert others.

`-00101111` => `11010000`
- ... but still 2 zeros.
  - `00000000`
  - `11111111`
- And what about arithmetics?

### Two's complement

... for negative integers, the absolute value of the integer is equal to "the magnitude of the complement of the (n-1)-bit binary pattern plus one" (hence called 2's complement).

`1B -13: ~(0 000 1101 - 1) -> ~(0 000 1100) -> (1 111 0011)`

`2B -13: ~(0 0000000 0000 1101 - 1) -> ~(0 0000000 0000 1100) -> (1 1111111 1111 0011)`

Sum:
```
    1111 0011
  + 0000 1101
    ---------
   10000 0000
```

In [5]:
b = c_byte(0b11110011)
s = c_int16(0b1111111111110011)
print(f'byte = {b.value}, int16 = {s.value}')

byte = -13, int16 = -13


#### Problems

1. What will be `-128` in 1-byte representation?
2. What is decimal for `01000001`?
3. What is decimal for `10000001`?

## Real numbers

In [6]:
f = -1.05e+2
print(f)

-105.0


We will speak about 1-precision numbers (`float`). Doubles are described by the same standard and differ only in positions.

For floating point value 4 bytes are used as follows:
1. The most significant bit is the **sign bit** (S), with 0 for positive numbers and 1 for negative numbers.
2. The following 8 bits represent **exponent** (E).
3. The remaining 23 bits represents **fraction** (F).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Float_example.svg/885px-Float_example.svg.png)

In [1]:
def bits_to_cfloat(bits):
    i = c_uint32(bits)                 # first we create unsigned 4-byte integer value
    ip = ctypes.addressof(i)           # then we take an address of this value
    fp = ctypes.cast(ip, ctypes.POINTER(c_float))  # and then we re-interpret the pointer as a pointer to 4-bytes float
    # bits will remain the same
    return fp[0]

### Example
`1 . 1000 0001 . 011 0000 0000 0000 0000 0000`, with:
- S = 1
- E = 1000 0001 = 129<sub>10</sub>
- F = 011 0000 0000 0000 0000 0000

1. In the normalized form, the actual fraction F is normalized with an implicit leading 1 in the form of `1.F`.
2. In normalized form, the actual exponent is `E-127` (-1024 for double)

Thus:

(-1) * (1 + 1/4 + 1/8) * 2<sup>(129 - 127)</sup> = -1.375 * 4 = -5.5
    

In [8]:
f = bits_to_cfloat(0b1_10000001_0110000_00000000_00000000)
print(f)

-5.5


Normalized form has a serious problem, with an implicit leading `1.` for the fraction, it cannot represent the number `0`.

### Special values and subnormal (denormalized) numbers

There are 2 special cases of exponent.

`E = 0000 0000`

1. If `F = 0`, then the number is considered ±0.
2. If `F > 0`, then this is **subnormal** number. Subnormal numbers processed differently: `value = S * 0.F * 2^(-126)`. There are also flush-to-zero (FTZ) and denormals-are-zero (DAZ) CPU flags.

E.g.

`1 . 0000 0000 . 011 0000 0000 0000 0000 0000`, with:
- S = 1
- E = 0000 0000
- F = 011 0000 0000 0000 0000 0000

`-0.375×2^(-126) ~ -4.4×10^(-39)`

In [9]:
f = bits_to_cfloat(0b1_0000_0000_011_0000_0000_0000_0000_0000)
print(f)

-4.408103815583578e-39


## Problem 2
Write a bit string for the biggest negative number. Check with code.

In [10]:
bits = 0b00000000000000000000000000000000
print(f'{bits_to_cfloat(bits):.50f}')

0.00000000000000000000000000000000000000000000000000


`E = 1111 1111`

1. If `F = 0000 0000` then the value is `inf`. Infinity can be both positive or negative.
2. If `F` is any other, then the value is *not-a-number* (NaN).
- ∞+(-∞)=NaN
- 0×∞=NaN
- ±0/±0=NaN
- ±∞/±∞=NaN
- sqrt(x) = NaN, где x<0

In [11]:
myminf = bits_to_cfloat(0b1_11111111_00000000000000000000000)
myinf = bits_to_cfloat(0b0_11111111_00000000000000000000000)
mynan = bits_to_cfloat(0b1_11111111_11111111111111111111111)
print(myinf, myminf, mynan)
print(myinf * 0)

inf -inf nan
nan


### Computations
- How many `float` values are there between `1.000(0)*2^N` and `1.111(1)*2^N`?
- What is their density?


-

-

-

-



In [ ]:
import matplotlib.pyplot as plt

def density(N):
    # TODO write your soultion here
    return 1

Ns = list(range(-10, 11))
Ds = [density(N) for N in Ns]

plt.plot(Ns, Ds)
plt.show()

In [12]:
x = 10. ** 17
print(f'{x:.1f}')
print(f'{x+1:.1f}')
print(f'{x+10:.1f}')
print(f'{x+100:.1f}')
print(f'{x+1000:.1f}')

100000000000000000.0
100000000000000000.0
100000000000000016.0
100000000000000096.0
100000000000000992.0


In [13]:
sum1 = 10000000000000.0
for i in range(2 ** 24):
    sum1 += 2 ** (-12)
print(f'{sum1:.40f}')

sum2 = 0.0
for i in range(2 ** 24):
    sum2 += 2 ** (-12)
sum2 += 10000000000000.0
print(f'{sum2:.40f}')

10000000000000.0000000000000000000000000000000000000000
10000000004096.0000000000000000000000000000000000000000
